# Experiment 084: EXACT Replication of ens-model Kernel

**Rationale**: Our exp_080 attempt got worse CV (0.010266 vs 0.008092). We may have missed exact hyperparameters.

**This experiment uses the EXACT parameters from the ens-model kernel:**
- CatBoost params for single: depth=3, lr=0.07, n_estimators=1050, l2_leaf_reg=3.5
- CatBoost params for full: depth=3, lr=0.06, n_estimators=1100, l2_leaf_reg=2.5
- XGBoost params: max_depth=4, lr=0.02, n_estimators=1000
- Ensemble weights: single (7:6), full (1:2)
- Probability normalization: clip to non-negative, divide by max(sum, 1.0)

In [1]:
import sys
import numpy as np
import pandas as pd
import torch
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
torch.set_default_dtype(torch.double)

print('Imports done')

Imports done


In [2]:
# Local data loading functions
def load_data(data_type):
    if data_type == "single_solvent":
        df = pd.read_csv('/home/data/catechol_single_solvent_yields.csv')
        X = df[['Residence Time', 'Temperature', 'SOLVENT NAME']]
        Y = df[['SM', 'Product 2', 'Product 3']]
    elif data_type == "full":
        df = pd.read_csv('/home/data/catechol_full_data_yields.csv')
        X = df[['Residence Time', 'Temperature', 'SOLVENT A NAME', 'SOLVENT B NAME', 'SolventB%']]
        Y = df[['SM', 'Product 2', 'Product 3']]
    return X, Y

def load_features(feature_type):
    if feature_type == 'spange_descriptors':
        return pd.read_csv('/home/data/spange_descriptors_lookup.csv', index_col=0)
    elif feature_type == 'drfps_catechol':
        return pd.read_csv('/home/data/drfps_catechol_lookup.csv', index_col=0)
    elif feature_type == 'fragprints':
        return pd.read_csv('/home/data/fragprints_lookup.csv', index_col=0)
    elif feature_type == 'acs_pca_descriptors':
        return pd.read_csv('/home/data/acs_pca_descriptors_lookup.csv', index_col=0)

print('Data functions defined')

Data functions defined


In [3]:
# Official CV split functions (DO NOT MODIFY)
from typing import Any, Generator

def generate_leave_one_out_splits(
    X: pd.DataFrame, Y: pd.DataFrame
) -> Generator[
    tuple[tuple[pd.DataFrame, pd.DataFrame], tuple[pd.DataFrame, pd.DataFrame]],
    Any,
    None,
]:
    for solvent in X["SOLVENT NAME"].unique():
        train_mask = X["SOLVENT NAME"] != solvent
        test_mask = X["SOLVENT NAME"] == solvent
        yield (
            (X[train_mask], Y[train_mask]),
            (X[test_mask], Y[test_mask]),
        )

def generate_leave_one_ramp_out_splits(
    X: pd.DataFrame, Y: pd.DataFrame
) -> Generator[
    tuple[tuple[pd.DataFrame, pd.DataFrame], tuple[pd.DataFrame, pd.DataFrame]],
    Any,
    None,
]:
    ramps = X["SOLVENT A NAME"].astype(str) + "_" + X["SOLVENT B NAME"].astype(str)
    for ramp in ramps.unique():
        train_mask = ramps != ramp
        test_mask = ramps == ramp
        yield (
            (X[train_mask], Y[train_mask]),
            (X[test_mask], Y[test_mask]),
        )

print('CV split functions defined')

CV split functions defined


In [4]:
# Feature priority for correlation filtering (from ens-model kernel)
def feature_priority(name: str) -> int:
    """Higher number = more important to keep during correlation filtering."""
    if name.startswith("spange_"):
        return 5
    if name.startswith("acs_"):
        return 4
    if name.startswith("drfps_"):
        return 3
    if name.startswith("frag_"):
        return 2
    return 0

def filter_correlated_features(df: pd.DataFrame, threshold: float = 0.90):
    """Drop columns that are highly correlated with any other column."""
    numeric_df = df.select_dtypes(include=[np.number])
    
    if numeric_df.shape[1] == 0:
        return df, []
    
    # Drop constant columns first
    std = numeric_df.std(axis=0)
    constant_cols = std[std == 0].index.tolist()
    if constant_cols:
        numeric_df = numeric_df.drop(columns=constant_cols)
    
    # Correlation matrix
    corr = numeric_df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool)).fillna(0.0)
    
    cols = upper.columns.tolist()
    to_drop = set()
    
    # Find all pairs with corr > threshold
    for i, col_i in enumerate(cols):
        for j in range(i + 1, len(cols)):
            col_j = cols[j]
            cval = upper.iloc[i, j]
            if cval > threshold:
                if col_i in to_drop or col_j in to_drop:
                    continue
                p_i = feature_priority(col_i)
                p_j = feature_priority(col_j)
                if p_i > p_j:
                    to_drop.add(col_j)
                elif p_j > p_i:
                    to_drop.add(col_i)
                else:
                    idx_i = df.columns.get_loc(col_i)
                    idx_j = df.columns.get_loc(col_j)
                    to_drop.add(col_i if idx_i > idx_j else col_j)
    
    all_to_drop = list(set(constant_cols).union(to_drop))
    df_filtered = df.drop(columns=all_to_drop, errors="ignore")
    
    return df_filtered, all_to_drop

print('Feature filtering functions defined')

Feature filtering functions defined


In [5]:
# Build combined solvent feature table (from ens-model kernel)
_SOLVENT_TABLE_CACHE = None

def build_solvent_feature_table(threshold: float = 0.90):
    """Build combined solvent feature table from multiple sources."""
    global _SOLVENT_TABLE_CACHE
    
    if _SOLVENT_TABLE_CACHE is not None:
        return _SOLVENT_TABLE_CACHE
    
    print(">>> Building solvent feature table...")
    
    sources = [
        "spange_descriptors",
        "acs_pca_descriptors",
        "drfps_catechol",
        "fragprints",
    ]
    
    dfs = []
    
    for src in sources:
        df_src = load_features(src).copy()
        
        if "SOLVENT NAME" not in df_src.columns:
            df_src = df_src.reset_index().rename(columns={"index": "SOLVENT NAME"})
        
        # Bit-table filtering for binary fingerprints
        if src in ["drfps_catechol", "fragprints"]:
            prefix = "drfps" if src == "drfps_catechol" else "frag"
            
            # Drop all-zero and all-one columns
            df_src = df_src.loc[:, (df_src != 0).any(axis=0)]
            df_src = df_src.loc[:, (df_src != 1).any(axis=0)]
            
            values = df_src.drop(columns={"SOLVENT NAME"}, errors='ignore')
            count = values.sum(axis=0).T
            drop_cols = count[count == 1].index
            df_src = df_src.drop(columns=drop_cols, errors='ignore')
            
            cols_to_rename = [c for c in df_src.columns if c != "SOLVENT NAME"]
            df_src = df_src.rename(columns={c: f"{prefix}_{c}" for c in cols_to_rename})
        else:
            if src == "spange_descriptors":
                prefix = "spange"
            elif src == "acs_pca_descriptors":
                prefix = "acs"
            else:
                prefix = src
            
            cols_to_rename = [c for c in df_src.columns if c != "SOLVENT NAME"]
            df_src = df_src.rename(columns={c: f"{prefix}_{c}" for c in cols_to_rename})
        
        dfs.append(df_src)
    
    # Merge all feature tables
    combined = reduce(lambda left, right: pd.merge(left, right, on="SOLVENT NAME", how="outer"), dfs)
    combined = combined.fillna(0.0)
    
    print(f"Combined shape before filtering: {combined.shape}")
    
    # Filter correlated features
    combined, dropped = filter_correlated_features(combined, threshold=threshold)
    
    print(f"Combined shape after filtering: {combined.shape}")
    print(f"Dropped {len(dropped)} features")
    
    combined = combined.set_index("SOLVENT NAME")
    _SOLVENT_TABLE_CACHE = combined
    
    return combined

print('Solvent table builder defined')

Solvent table builder defined


In [6]:
# Numeric feature engineering (from ens-model kernel)
def add_numeric_features(X_numeric: pd.DataFrame) -> pd.DataFrame:
    """Add engineered numeric features - EXACTLY as in ens-model kernel."""
    X_num = X_numeric.copy()
    cols = set(X_num.columns)
    
    if {"Temperature", "Residence Time"} <= cols:
        # Convert Temperature to Kelvin (CRITICAL!)
        X_num["Temperature"] = X_num["Temperature"] + 273.15
        
        T = X_num["Temperature"]
        rt = X_num["Residence Time"]
        
        # Interaction term
        X_num["T_x_RT"] = T * rt
        
        # Log transformation
        X_num["RT_log"] = np.log(rt + 1e-6)
        
        # Inverse temperature (Arrhenius-like)
        X_num["T_inv"] = 1 / T
        
        # Scaled residence time
        X_num["RT_scaled"] = rt / rt.mean()
    
    return X_num

print('Numeric feature engineering defined')

Numeric feature engineering defined


In [7]:
# Featurizers (from ens-model kernel)
class PrecomputedFeaturizer:
    """Featurizer for single solvent data."""
    
    def __init__(self):
        self.solvent_table = build_solvent_feature_table(threshold=0.90)
    
    def featurize(self, X: pd.DataFrame) -> torch.Tensor:
        # Numeric features
        X_numeric = X[["Residence Time", "Temperature"]].copy()
        X_numeric = add_numeric_features(X_numeric)
        
        # Solvent features
        solvent_names = X["SOLVENT NAME"].values
        solvent_feats = self.solvent_table.loc[solvent_names].values
        
        # Combine
        combined = np.hstack([X_numeric.values, solvent_feats])
        combined = np.nan_to_num(combined, nan=0.0, posinf=1e6, neginf=-1e6)
        
        return torch.tensor(combined, dtype=torch.double)


class PrecomputedFeaturizerMixed:
    """Featurizer for mixed solvent data."""
    
    def __init__(self):
        self.solvent_table = build_solvent_feature_table(threshold=0.90)
    
    def featurize(self, X: pd.DataFrame) -> torch.Tensor:
        # Numeric features
        X_numeric = X[["Residence Time", "Temperature"]].copy()
        X_numeric = add_numeric_features(X_numeric)
        
        # Solvent features (linear mixing)
        solvent_a = X["SOLVENT A NAME"].values
        solvent_b = X["SOLVENT B NAME"].values
        ratio_b = X["SolventB%"].values / 100.0
        
        feats_a = self.solvent_table.loc[solvent_a].values
        feats_b = self.solvent_table.loc[solvent_b].values
        
        # Linear mixing
        solvent_feats = (1 - ratio_b.reshape(-1, 1)) * feats_a + ratio_b.reshape(-1, 1) * feats_b
        
        # Add ratio as feature
        X_numeric["SolventB_ratio"] = ratio_b
        
        # Combine
        combined = np.hstack([X_numeric.values, solvent_feats])
        combined = np.nan_to_num(combined, nan=0.0, posinf=1e6, neginf=-1e6)
        
        return torch.tensor(combined, dtype=torch.double)

print('Featurizers defined')

Featurizers defined


In [8]:
# CatBoost Model with EXACT params from ens-model kernel
from catboost import CatBoostRegressor

class CatBoostModel:
    """CatBoost model with EXACT params from ens-model kernel."""
    
    def __init__(self, data: str = "single", random_state: int = 42):
        self.data_mode = data
        self.random_state = random_state
        
        if data == "single":
            self.smiles_featurizer = PrecomputedFeaturizer()
            # EXACT params from ens-model kernel for single solvent
            self.cat_params = dict(
                random_seed=random_state,
                loss_function="MultiRMSE",
                depth=3,
                learning_rate=0.07,
                n_estimators=1050,
                l2_leaf_reg=3.5,
                bootstrap_type="Bayesian",
                bagging_temperature=0.225,
                grow_policy="SymmetricTree",
                rsm=0.75,
                verbose=False,
            )
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed()
            # EXACT params from ens-model kernel for full/mixed
            self.cat_params = dict(
                random_seed=random_state,
                loss_function="MultiRMSE",
                depth=3,
                learning_rate=0.06,
                n_estimators=1100,
                l2_leaf_reg=2.5,
                bootstrap_type="Bayesian",
                bagging_temperature=0.25,
                grow_policy="SymmetricTree",
                rsm=0.75,
                verbose=False,
            )
        
        self.model = None
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        X_tensor = self.smiles_featurizer.featurize(train_X)
        X_np = X_tensor.detach().cpu().numpy()
        Y_np = train_Y.values
        
        # CatBoost with MultiRMSE handles multi-output natively
        self.model = CatBoostRegressor(**self.cat_params)
        self.model.fit(X_np, Y_np)
    
    def predict(self, X):
        if self.model is None:
            raise RuntimeError("CatBoostModel not trained")
        
        X_tensor = self.smiles_featurizer.featurize(X)
        X_np = X_tensor.detach().cpu().numpy()
        
        out = self.model.predict(X_np)
        
        # PROBABILITY NORMALIZATION (from ens-model kernel)
        out = np.clip(out, a_min=0.0, a_max=None)
        if out.ndim > 1 and out.shape[1] > 1:
            totals = out.sum(axis=1, keepdims=True)
            divisor = np.maximum(totals, 1.0)
            out = out / divisor
        
        return torch.tensor(out, dtype=torch.double)

print('CatBoost model defined')

CatBoost model defined


In [9]:
# XGBoost Model with EXACT params from ens-model kernel
import xgboost as xgb

class XGBModel:
    """XGBoost model with EXACT params from ens-model kernel."""
    
    def __init__(self, data: str = "single", random_state: int = 42):
        self.data_mode = data
        self.random_state = random_state
        
        if data == "single":
            self.smiles_featurizer = PrecomputedFeaturizer()
            # EXACT params from ens-model kernel for single
            self.xgb_params = dict(
                random_state=random_state,
                objective="reg:squarederror",
                tree_method="hist",
                subsample=0.5,
                reg_lambda=0.6,
                reg_alpha=0.0,
                n_estimators=1000,
                min_child_weight=1,
                max_depth=4,
                max_delta_step=1,
                learning_rate=0.02,
                grow_policy="depthwise",
                gamma=0.0,
            )
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed()
            # EXACT params from ens-model kernel for full
            self.xgb_params = dict(
                random_state=random_state,
                objective="reg:squarederror",
                tree_method="hist",
                subsample=0.6,
                reg_lambda=0.5,
                reg_alpha=0.0,
                n_estimators=1100,
                min_child_weight=1,
                max_depth=4,
                max_delta_step=1,
                learning_rate=0.02,
                grow_policy="depthwise",
                gamma=0.0,
            )
        
        self.models = None
        self.n_targets = None
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        X_tensor = self.smiles_featurizer.featurize(train_X)
        X_np = X_tensor.detach().cpu().numpy()
        Y_np = train_Y.values
        self.n_targets = Y_np.shape[1]
        
        # Train one XGBoost per target
        self.models = []
        for t in range(self.n_targets):
            m = xgb.XGBRegressor(**self.xgb_params)
            m.fit(X_np, Y_np[:, t])
            self.models.append(m)
    
    def predict(self, X):
        if self.models is None:
            raise RuntimeError("XGBModel not trained")
        
        X_tensor = self.smiles_featurizer.featurize(X)
        X_np = X_tensor.detach().cpu().numpy()
        
        preds_list = [m.predict(X_np) for m in self.models]
        out = np.column_stack(preds_list)
        
        # PROBABILITY NORMALIZATION (from ens-model kernel)
        out = np.clip(out, a_min=0.0, a_max=None)
        if out.shape[1] > 1:
            totals = out.sum(axis=1, keepdims=True)
            divisor = np.maximum(totals, 1.0)
            out = out / divisor
        
        return torch.tensor(out, dtype=torch.double)

print('XGBoost model defined')

XGBoost model defined


In [10]:
# Ensemble Model with EXACT weights from ens-model kernel
class EnsembleModel:
    """Weighted ensemble of CatBoost and XGBoost with EXACT weights from ens-model kernel."""
    
    def __init__(self, data: str = "single", verbose: bool = False):
        self.data_mode = data
        self.verbose = verbose
        
        # EXACT weights from ens-model kernel
        if data == "single":
            cat_weight = 7.0
            xgb_weight = 6.0
        else:
            cat_weight = 1.0
            xgb_weight = 2.0
        
        # Normalize ensemble weights
        w_sum = cat_weight + xgb_weight
        self.cat_weight = cat_weight / w_sum
        self.xgb_weight = xgb_weight / w_sum
        
        # Initialize base models
        self.cat_model = CatBoostModel(data=data)
        self.xgb_model = XGBModel(data=data)
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        self.cat_model.train_model(train_X, train_Y)
        self.xgb_model.train_model(train_X, train_Y)
    
    def predict(self, X):
        cat_pred = self.cat_model.predict(X)
        xgb_pred = self.xgb_model.predict(X)
        
        out = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
        
        return out

print('EnsembleModel defined')

EnsembleModel defined


In [11]:
# Run CV for single solvent data
import tqdm

X, Y = load_data("single_solvent")
print(f"Single solvent data: {len(X)} samples, {len(X['SOLVENT NAME'].unique())} solvents")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
fold_mses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = EnsembleModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    # Calculate fold MSE
    fold_mse = np.mean((predictions_np - test_Y.values) ** 2)
    fold_mses.append(fold_mse)
    
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"\nSingle solvent CV MSE: {np.mean(fold_mses):.6f} ± {np.std(fold_mses):.6f}")

Single solvent data: 656 samples, 24 solvents


  0%|          | 0/24 [00:00<?, ?it/s]

>>> Building solvent feature table...
Combined shape before filtering: (26, 114)
Combined shape after filtering: (26, 67)
Dropped 47 features


  4%|▍         | 1/24 [00:01<00:39,  1.70s/it]

  8%|▊         | 2/24 [00:02<00:30,  1.40s/it]

...

 92%|█████████▏| 22/24 [00:25<00:02,  1.11s/it]

 96%|█████████▌| 23/24 [00:26<00:01,  1.10s/it]

100%|██████████| 24/24 [00:27<00:00,  1.10s/it]

100%|██████████| 24/24 [00:27<00:00,  1.15s/it]


Single solvent CV MSE: 0.009046 ± 0.008474


In [12]:
# Run CV for full (mixture) data
X, Y = load_data("full")
print(f"Full data: {len(X)} samples")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []
fold_mses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = EnsembleModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    # Calculate fold MSE
    fold_mse = np.mean((predictions_np - test_Y.values) ** 2)
    fold_mses.append(fold_mse)
    
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

submission_full_data = pd.DataFrame(all_predictions)
print(f"\nFull data CV MSE: {np.mean(fold_mses):.6f} ± {np.std(fold_mses):.6f}")

Full data: 1227 samples


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:01<00:22,  1.86s/it]

 15%|█▌        | 2/13 [00:03<00:20,  1.84s/it]

 23%|██▎       | 3/13 [00:05<00:18,  1.82s/it]

...

 85%|████████▍ | 11/13 [00:20<00:03,  1.82s/it]

 92%|█████████▏| 12/13 [00:21<00:01,  1.86s/it]

100%|██████████| 13/13 [00:23<00:00,  1.89s/it]

100%|██████████| 13/13 [00:23<00:00,  1.84s/it]


Full data CV MSE: 0.009501 ± 0.004401


In [ ]:
# Combine and save submission
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"

print(f"Submission shape: {submission.shape}")

# Save
submission.to_csv("/home/submission/submission.csv", index=True)
print(f"\nSubmission saved to /home/submission/submission.csv")

# Verify
submission_check = pd.read_csv("/home/submission/submission.csv")
print(f"\nSubmission rows: {len(submission_check)}")

# Check prediction ranges and row sums
target_cols = ['target_1', 'target_2', 'target_3']
for col in target_cols:
    print(f"{col}: min={submission_check[col].min():.4f}, max={submission_check[col].max():.4f}")

# Check row sums
row_sums = submission_check[target_cols].sum(axis=1)
print(f"\nRow sums: min={row_sums.min():.4f}, max={row_sums.max():.4f}, mean={row_sums.mean():.4f}")

In [ ]:
# Calculate overall CV score
print("="*50)
print("EXPERIMENT 084 COMPLETE")
print("="*50)
print(f"\nKey techniques from ens-model kernel:")
print("1. Combined features from multiple sources (spange, acs_pca, drfps, fragprints)")
print("2. Correlation-based feature filtering (threshold=0.90)")
print("3. Numeric feature engineering (T_inv, RT_log, T_x_RT, RT_scaled with T in Kelvin)")
print("4. CatBoost + XGBoost ensemble with EXACT params")
print("5. EXACT ensemble weights: single (7:6), full (1:2)")
print("6. Probability normalization: clip to non-negative, divide by max(sum, 1.0)")